## Wrangle report

### Gathering the data

To gather data from different sources such as direct download from storage via HTTP, twitter API and csv file I have used "requests, "tweepy" and pandas libraries.    

a. Provided archive appeared to be comma separated file so loading did take much effort it was done via pandas read_csv method with default separator.    

b. To utilize Twitter API I used to work with tweepy method "get_statuses" which returns tweet's info by id. Each post and its data stored as separate JSON line in tweet_json.txt. About 10 provided ids were not found among existed in twitter.    

Those are :
- 888202515573088257
- 873697596434513921
- 869988702071779329
- 866816280283807744
- 861769973181624320
- 842892208864923648
- 837012587749474308
- 827228250799742977
- 802247111496568832
- 775096608509886464

c. Third set came from http request made to provided URL, I stored result of HTTP request as binary data to file. That was tab separated file. To load that as data frame all we need just to change default separator to \t one. 

Several columns had have not appropriate format and it was decided to **convert tweet_id** to int64, **timestamp and retweeted_status_timestamp** to date and time. Before converting to datetime object I cut off ending "0000".

Next step was to check in text column whether **numerator and denominator** data was properly parsed. It appeared many rows had a problems: zero in denominator, not properly parsed decimal values as 9.75, too big num and denom (those I scaled down proportionally e.x 196/140 = 14/10), in some cases marks were taken from wrong positions in sentences were also know patterns as 50/50, 24/7 were used. I did not write generic function as there were variety of cases and function was not going to be super flexible anyway so i did fix them in semi manual way (array of indexes to be fixed and array of values to change problem values to). All denominators I set to 10 after numerator problems were revealed. Then I divide num/enum and stored values in rating column. Several outliers spotted 66.6, 142, etc.

After num and denum I looked at **name** column via value_counts function and found that there were lots of names comprised of one letter "A", or looking like article "the", "an" those were looking like parsing errors.
Names were misspelled as within description dogs referred by "normal" names or that was just parsing error which is likely for names "a", "the" "an" words were cut from the text from incorrect position. Among "an" names there was only one real row (2204, Berta), among "the" real names were not found. Also I had to capitalize name so they look all like. 755 rows were NaN. All those not names were dropped or set to NaN. Among "A" I searched whether row contained "named" pattern if so names were extracted and set to name column. After that has been done 36 rows left, the only row from these 36 was real name "Daryl" so I fixed that manually and did drop others.

**"source"** was erased as I found that data not valuable enough.

**'doggo', 'floofer', 'pupper', 'puppo'** columns were transformed into values for newly introduced "type" feature. Anyway lots of None there were as well, I did check text once again and found few more not parsed value but did not fix that as it would not help anyway. 

**expanded_urls** - several different resources were referenced within cell among them twitter, gofundme (?) probably I misspelled, and twitter *littlewiewel that gives 404.* Dupes within cells were spotted and also outside the cells. Not only pictures were stored but in rows but also videos. I filtered out all the rows which are not pictures from twitter. After cleaning I met second portion of dupes from different cells and with different tweet_id's. Those were retweeted posts.. filtered out.

**retweeted_status_user_id** belongs to retweets all non nulls were deleted and column then dropped.

**non-recognizable data** that existed in twitter archive but did not in recognition data set was removed.

**text** should have been split into comment, mark and short url. Since we already had numerator and denominator in our data set we could keep only text and delete redundant info. It looks like in text column there was only one link when "expanded_links" columns had one or more links even to external resources.
We could get text extracted using unique regex pattern to track all possible positions for text, link and marks but that would not make much sense since we already had names, marks and links in our set. It seems like all we needed was to parse first part of text till the rating position and cut the rest coming after that position since it was related to the mark itself rather than dog description. After cleaning rows with problems were still existed in some places. To fix completely we would need to re-iterate..
 
**dupes in recognition set** were removed, we used id's from archive set to track them down.

**merge** Firstly I merged twitter archive and recognition sets as they already had equal amount of ids. After that I renamed range of columns to new names.

**breeds** Many rows had different format as word_Word, Word_word, or word word, so i split them by underscore value, formated every single thing to lower case and capitalized each word.

**dowloaded data** very complex json, decided to keep only retweeted count, favourite and id to merge later. I had to rename id to tweet_id as in other columns and covert to int64.

**merge** Once again by id and got final data set which was saved on disk as csv file.